### Handling Missing Values
+ Dropping Columns
+ Dropping Rows
+ Various Parameter in Dropping functionalities
+ Handling Missing values by Mean, Median and Mode

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab/ssafy_second_pjt/spark

/content/drive/MyDrive/Colab/ssafy_second_pjt/spark


In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=47359ecf160d082e0773cb1916c1934057369440f37073de8a40f9883736c3af
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tutorial_03').getOrCreate()

In [5]:
sdf = spark.read.csv('test2.csv', header=True, inferSchema=True)
sdf.show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|   Krish|  31|        10| 50000|
|Sudhansh|  30|         7| 40000|
|   Sunny|  29|         5| 32000|
|    Paul|  24|         3| 28000|
|       J|  22|         2| 27000|
|    Park|  41|        14| 60000|
|       Q|null|      null| 40000|
|    null|  34|        12|  null|
|    null|  37|      null|  null|
+--------+----+----------+------+



In [7]:
sdf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
# null 있는 record 모두 삭제
sdf.dropna().show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 50000|
|Sudhansh| 30|         7| 40000|
|   Sunny| 29|         5| 32000|
|    Paul| 24|         3| 28000|
|       J| 22|         2| 27000|
|    Park| 41|        14| 60000|
+--------+---+----------+------+



In [9]:
# 전체가 null 인 record 삭제
sdf.dropna(how='all').show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|   Krish|  31|        10| 50000|
|Sudhansh|  30|         7| 40000|
|   Sunny|  29|         5| 32000|
|    Paul|  24|         3| 28000|
|       J|  22|         2| 27000|
|    Park|  41|        14| 60000|
|       Q|null|      null| 40000|
|    null|  34|        12|  null|
|    null|  37|      null|  null|
+--------+----+----------+------+



In [18]:
# non-null 값 개수가 thresh 미만이면 삭제
sdf.dropna(how='any',thresh=3).show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 50000|
|Sudhansh| 30|         7| 40000|
|   Sunny| 29|         5| 32000|
|    Paul| 24|         3| 28000|
|       J| 22|         2| 27000|
|    Park| 41|        14| 60000|
+--------+---+----------+------+



In [19]:
sdf.dropna(how='any', subset=['Experience']).show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 50000|
|Sudhansh| 30|         7| 40000|
|   Sunny| 29|         5| 32000|
|    Paul| 24|         3| 28000|
|       J| 22|         2| 27000|
|    Park| 41|        14| 60000|
|    null| 34|        12|  null|
+--------+---+----------+------+



In [22]:
# 결측치 채우기
sdf.fillna({'Name': 'anonymous',  'Age': '99',    'Experience': '10' , 'Salary':'3'}).show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 50000|
| Sudhansh| 30|         7| 40000|
|    Sunny| 29|         5| 32000|
|     Paul| 24|         3| 28000|
|        J| 22|         2| 27000|
|     Park| 41|        14| 60000|
|        Q| 99|        10| 40000|
|anonymous| 34|        12|     3|
|anonymous| 37|        10|     3|
+---------+---+----------+------+



In [26]:
# 평균으로 결측치 채우기
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age','Experience', 'Salary'],
    outputCols=['{}_imputed'.format(c) for c in ['Age','Experience', 'Salary']]
).setStrategy("mean")  # mean, median, mode

In [27]:
imputer.fit(sdf).transform(sdf).show()

+--------+----+----------+------+-----------+------------------+--------------+
|    Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+--------+----+----------+------+-----------+------------------+--------------+
|   Krish|  31|        10| 50000|         31|                10|         50000|
|Sudhansh|  30|         7| 40000|         30|                 7|         40000|
|   Sunny|  29|         5| 32000|         29|                 5|         32000|
|    Paul|  24|         3| 28000|         24|                 3|         28000|
|       J|  22|         2| 27000|         22|                 2|         27000|
|    Park|  41|        14| 60000|         41|                14|         60000|
|       Q|null|      null| 40000|         22|                 2|         40000|
|    null|  34|        12|  null|         34|                12|         40000|
|    null|  37|      null|  null|         37|                 2|         40000|
+--------+----+----------+------+-------